<a href="https://colab.research.google.com/github/vitchierath/Gen_Ai_miniprojects/blob/main/multiplepdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [14]:
import os
import PyPDF2
import sys
import io
import uuid
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Detect if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

# Check for dependencies
try:
    if IN_COLAB:
        from google.colab import files, drive
    else:
        import ipywidgets as widgets
        from IPython.display import display
except ImportError:
    print("ipywidgets not installed. File upload in Jupyter requires ipywidgets.")
    print("Install with: pip install ipywidgets")
    sys.exit(1)

required_packages = [
    'PyPDF2', 'langchain', 'langchain_huggingface', 'chromadb',
    'sentence_transformers', 'transformers', 'torch', 'huggingface_hub'
]
missing_packages = []
for pkg in required_packages:
    try:
        __import__(pkg)
    except ImportError:
        missing_packages.append(pkg)

if missing_packages:
    print("Missing required packages:", ", ".join(missing_packages))
    print("Install with:")
    print("pip install", " ".join(missing_packages))
    if IN_COLAB:
        print("Or in Colab, run: !pip install", " ".join(missing_packages))
    sys.exit(1)


def mount_google_drive():
    """Mount Google Drive in Colab."""
    if not IN_COLAB:
        return False
    try:
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
        return True
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return False

def extract_text_from_pdf_file(pdf_file, is_uploaded=False):
    """Extract text from a single PDF file."""
    try:
        if is_uploaded:
            pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file))
        else:
            with open(pdf_file, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)

        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text
    except Exception as e:
        print(f"Error processing {pdf_file if not is_uploaded else 'uploaded file'}: {e}")
        return ""

def upload_files_colab():
    """Handle file uploads in Google Colab."""
    if not IN_COLAB:
        raise EnvironmentError("File upload is only supported in Google Colab.")
    print("Please upload your PDF files:")
    uploaded = files.upload()
    return uploaded

def upload_files_jupyter():
    """Handle file uploads in Jupyter using ipywidgets."""
    if IN_COLAB:
        raise EnvironmentError("Jupyter upload not applicable in Colab.")
    uploader = widgets.FileUpload(accept='.pdf', multiple=True)
    display(uploader)
    print("Upload PDFs and then enter 'process' to continue.")
    return uploader

def process_uploaded_files(uploaded):
    """Process uploaded files from Colab or Jupyter."""
    texts = []
    if IN_COLAB:
        for filename, file_content in uploaded.items():
            if not filename.lower().endswith('.pdf'):
                print(f"Skipping {filename}: Not a PDF file")
                continue
            text = extract_text_from_pdf_file(file_content, is_uploaded=True)
            if text.strip():
                texts.append(text)
            else:
                print(f"Warning: No text extracted from {filename}")
    else:
        for filename, file_data in uploaded.value.items():
            if not filename.lower().endswith('.pdf'):
                print(f"Skipping {filename}: Not a PDF file")
                continue
            text = extract_text_from_pdf_file(file_data['content'], is_uploaded=True)
            if text.strip():
                texts.append(text)
            else:
                print(f"Warning: No text extracted from {filename}")
    return texts

def extract_text_from_pdfs(source, is_directory=False):
    """Extract text from PDFs (directory or uploaded files)."""
    texts = []

    if is_directory:
        pdf_directory = Path(source)
        if not pdf_directory.exists():
            raise FileNotFoundError(f"Directory {source} does not exist.")

        for pdf_file in pdf_directory.glob("*.pdf"):
            text = extract_text_from_pdf_file(pdf_file, is_uploaded=False)
            if text.strip():
                texts.append(text)
            else:
                print(f"Warning: No text extracted from {pdf_file}")
    else:
        texts = process_uploaded_files(source)

    return texts

def create_vector_store(texts):
    """Create a vector store from extracted texts."""
    if not texts:
        raise ValueError("No valid text extracted from PDFs")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    documents = text_splitter.create_documents(texts)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=f"/tmp/chroma_{uuid.uuid4()}")

    return vectorstore

def setup_qa_pipeline():
    """Set up the question-answering pipeline."""
    model_name = "distilbert-base-cased-distilled-squad"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=-1)
        return qa_pipeline
    except Exception as e:
        print(f"Error setting up QA pipeline: {e}")
        raise

def setup_summarization_pipeline():
    """Set up the summarization pipeline."""
    model_name = "facebook/bart-large-cnn"
    try:
        summarizer = pipeline("summarization", model=model_name, device=-1)
        return summarizer
    except Exception as e:
        print(f"Error setting up summarization pipeline: {e}")
        raise

def generate_summary(texts, summarizer):
    """Generate a summary of the PDF content."""
    # Combine all texts into a single string
    full_text = " ".join(texts)
    if not full_text.strip():
        return "No valid text found in the PDFs to summarize."

    # Split into chunks for summarization (BART has token limits)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_text(full_text)

    # Summarize each chunk and combine
    summaries = []
    for chunk in chunks:
        try:
            summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
            summaries.append(summary)
        except Exception as e:
            print(f"Error summarizing chunk: {e}")
            summaries.append("Failed to summarize this chunk.")

    # Combine summaries into a final summary
    final_summary = " ".join(summaries)
    if len(final_summary) > 1000:
        # Summarize the combined summaries if too long
        try:
            final_summary = summarizer(final_summary, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
        except Exception as e:
            print(f"Error summarizing combined summaries: {e}")
            final_summary = final_summary[:1000] + "... (truncated)"

    return final_summary

def answer_question(question, vectorstore, qa_pipeline):
    """Answer a question using the vector store and QA pipeline."""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    docs = retriever.get_relevant_documents(question)

    context = " ".join([doc.page_content for doc in docs])
    if not context.strip():
        return "No relevant content found in the PDFs.", []

    try:
        result = qa_pipeline(question=question, context=context)
        answer = result['answer']
    except Exception as e:
        print(f"Error in QA pipeline: {e}")
        answer = "Failed to generate an answer due to pipeline error."

    return answer, docs

def main():
    """Main function to process PDFs, generate a summary, and answer questions."""
    if IN_COLAB:
        print("Choose PDF input method:")
        print("1. Upload PDF files")
        print("2. Load PDFs from Google Drive directory")

        choice = input("Enter 1, 2: ").strip()
    else:
        print("Choose PDF input method:")
        print("1. Upload PDF files (Jupyter only)")
        print("2. Load PDFs from local directory")

        choice = input("Enter 1, 2 ").strip()

    texts = []
    if choice == "1":
        if IN_COLAB:
            uploaded = upload_files_colab()
        else:
            try:
                uploader = upload_files_jupyter()
                user_input = input("Enter 'process' to proceed: ").strip().lower()
                if user_input != 'process':
                    return "Aborted. Please rerun and enter 'process' after uploading.", []
                uploaded = uploader
            except EnvironmentError:
                return "File upload is only supported in Jupyter or Colab environments.", []
        texts = extract_text_from_pdfs(uploaded, is_directory=False)
    elif choice == "2":
        if IN_COLAB:
            if mount_google_drive():
                pdf_directory = input("Enter the Google Drive directory path (e.g., /content/drive/MyDrive/pdfs): ").strip()
                try:
                    texts = extract_text_from_pdfs(pdf_directory, is_directory=True)
                except FileNotFoundError as e:
                    return str(e), []
            else:
                return "Failed to mount Google Drive.", []
        else:
            pdf_directory = input("Enter the local directory path (e.g., ./pdfs): ").strip()
            try:
                texts = extract_text_from_pdfs(pdf_directory, is_directory=True)
            except FileNotFoundError as e:
                return str(e), []
    else:
        return "Invalid choice. Please select 1 or 2.", []

    if not texts:
        return "No valid PDF content found. Please upload PDFs or specify a valid directory.", []

    # Generate summary
    print("\nGenerating summary of content...")
    summarizer = setup_summarization_pipeline()
    summary = generate_summary(texts, summarizer)
    print("Summary:")
    print(summary)

    # Create vector store for QA
    vectorstore = create_vector_store(texts)

    # Setup QA pipeline
    qa_pipeline = setup_qa_pipeline()

    # Chat-like question loop
    print("\nAsk follow-up questions (type 'exit' to stop):")
    while True:
        question = input("You: ").strip()
        if question.lower() == 'exit':
            print("Me: Goodbye!")
            break
        if not question:
            print("Me: Please enter a valid question.")
            continue

        answer, sources = answer_question(question, vectorstore, qa_pipeline)
        print(f"Me: {answer}")
        print("\nSources:")
        for i, doc in enumerate(sources, 1):
            print(f"{i}. {doc.page_content[:200]}...")

if __name__ == "__main__":
    try:
        result = main()
        if isinstance(result, tuple):
            answer, sources = result
            print(f"Error: {answer}")
            print("\nSources:")
            for i, doc in enumerate(sources, 1):
                print(f"{i}. {doc.page_content[:200]}...")
    except Exception as e:
        print(f"Error: {e}")